In [2]:
# default_exp CUFile

# ShiftClickUp

> Shift and Score ClickUp Tasks.

In [ ]:
#export
import urllib.parse, urllib.request
import json
from urllib.parse import urlencode, quote_plus
import http.client
import time
from datetime import datetime, timedelta


class CU:   
    def __init__(self, _apikey):

        self.apikey = _apikey
        teamdata = self.urlrequest("https://api.clickup.com/api/v2/team")
        teamid = teamdata["teams"][0]["id"]
        spacedata = self.urlrequest("https://api.clickup.com/api/v2/team/" + teamid + "/space")
        spaceid = spacedata["spaces"][0]["id"]
        self.folderdata = self.urlrequest("https://api.clickup.com/api/v2/space/" + spaceid + "/folder")

    
    def urlrequest(self, url, param=None):
        req = urllib.request.Request(url)
        req.add_header("Authorization",self.apikey)
        resp = urllib.request.urlopen(req, timeout=200)
        responsedata = resp.read()
        data = json.loads(responsedata)   
        return data

   
    def updatescore(self, fieldid, taskid, value):
            conn = http.client.HTTPSConnection("api.clickup.com")

            payload = "{\r\n  \"value\": " + value + "\r\n}"

            headers = {
                'authorization': self.apikey,
                'content-type': "application/json",
                'cache-control': "no-cache"
                }

            conn.request("POST", "/api/v2/task/" + taskid + "/field/" + fieldid + "/", payload, headers)

            res = conn.getresponse()
            data = res.read()
            return data.decode("utf-8")


    def updatepriority(self, taskid, value):
        conn = http.client.HTTPSConnection("api.clickup.com")

        payload = "{\r\n  \"priority\": " + value + "\r\n}"

        headers = {
            'authorization': self.apikey,
            'content-type': "application/json",
            'cache-control': "no-cache"
            }

        conn.request("PUT", "/api/v2/task/" + taskid + "/", payload, headers)

        res = conn.getresponse()
        data = res.read()
        return data.decode("utf-8")

    def shiftstartandduedate(self, taskid, startvalue, duevalue, aantaldagen):
        startvalue = startvalue + ((60 * 1000 * 60 * 24) * aantaldagen)
        duevalue = duevalue + ((60 * 1000 * 60 * 24) * aantaldagen)
        conn = http.client.HTTPSConnection("api.clickup.com")

        payload = "{\r\n  \"start_date\": " + str(startvalue) + ",\r\n  \"due_date\": " + str(duevalue) + "\r\n}"

        headers = {
            'authorization': self.apikey,
            'content-type': "application/json",
            'cache-control': "no-cache"
            }

        conn.request("PUT", "/api/v2/task/" + taskid + "/", payload, headers)

        res = conn.getresponse()
        data = res.read()
        return data.decode("utf-8")

    def updatetasks(self, keuze, dagen):        
        t = int(time.time())
        for folder in self.folderdata["folders"]: 
            folderid = folder["id"]
            print(folder["name"])
            listdata = self.urlrequest("https://api.clickup.com/api/v2/folder/" + folderid + "/list")
            for list in listdata["lists"]: 
                listid = list["id"]
                taskdata = self.urlrequest("https://api.clickup.com/api/v2/list/" + listid + "/task")
                for task in taskdata["tasks"]:

                    try:
                        if keuze == "score":                            
                            #print(task["name"])
                            #print('1:' + task["name"])
                            #aantal uur, deze is goed
                            #print('2.1:' + str(((t * 1000) - int(task["date_updated"])) / 1000 / 60 / 60 < 8  ) )
                            #print('3:' + str(task["date_updated"]))
                            #print('4:' + str(time.time()))
                            if  (task["custom_fields"][4].get('value') is None) or (((t * 1000) - int(task["date_updated"])) / 1000 / 60 / 60 / 24 < dagen) :
                                score = ( ((5 - int(task["custom_fields"][0]["value"])) +
                                 (2*  int(task["custom_fields"][1]["value"]) ) +
                                (2 * int(task["custom_fields"][2]["value"])) +
                                int(task["custom_fields"][3]["value"])) / 6) 
                                print('score updated for: ')
                                print(task["name"])
                                self.updatescore(task["custom_fields"][4]["id"],task["id"],str(score))
                        if keuze == "shift":
                            if task["status"]["status"] == "Open":                            
                                self.shiftstartandduedate(task["id"], int(task["start_date"]), int(task["due_date"]), dagen)
                        elif keuze == "priority" and ((task["priority"] == None) or (((t * 1000) - int(task["date_updated"])) / 1000 / 60 / 60 / 24 < dagen)):
                            score = ( ((5 - int(task["custom_fields"][0]["value"])) +
                                 (2*  int(task["custom_fields"][1]["value"]) ) +
                                (2 * int(task["custom_fields"][2]["value"])) +
                                int(task["custom_fields"][3]["value"])) / 6) 
                            print("prio updated for: ")
                            print(task["name"])
                            if score >= 4:
                                self.updatepriority(task["id"],"1")
                            elif score >= 3:
                                self.updatepriority(task["id"],"2")
                            elif score >= 2:
                                self.updatepriority(task["id"],"3")
                            else:
                                self.updatepriority(task["id"],"4")
                    except Exception as e:
                        print(e)
                        print(task["name"])